Importing `SQLServer`

In [2]:
from Database import *

Making instance and connecting to SQL Server via Data Source Name with windows authentication

In [3]:
sql = SQLServer(dsn='DSN_TEST')

In [4]:
sql.connect()

Connection established with DSN_TEST
Using Database: master
Use set_database(database) method to use a specific database


Using `list_database` method to show existing databases

In [5]:
databases = sql.list_database()
databases[:4]

['master', 'tempdb', 'model', 'msdb']

Using `set_database` method to access desired database

In [6]:
sql.set_database('AdventureWorks2017')

Database AdventureWorks2017 accessed


`list_tables` method has two possible output options: __dataframe__ or __list__

as dataframe:

In [15]:
tables_df = sql.list_tables(output='dataframe')
tables.head()

,TABLE_NAME,TABLE_SCHEMA,TABLE_TYPE
0,EmployeePayHistory,HumanResources,BASE TABLE
1,SalesOrderHeaderSalesReason,Sales,BASE TABLE
2,SalesPerson,Sales,BASE TABLE
3,Illustration,Production,BASE TABLE
4,JobCandidate,HumanResources,BASE TABLE


as list:

In [16]:
tables_list = sql.list_tables(output='list')
tables_list[:5]

['EmployeePayHistory',
 'SalesOrderHeaderSalesReason',
 'SalesPerson',
 'Illustration',
 'JobCandidate']

Using `detail_table` method to show information about a specific table

In [8]:
sql.detail_table('SalesPerson')

,COLUMN_NAME,DATA_TYPE,IS_NULLABLE
0,BusinessEntityID,int,NO
1,TerritoryID,int,YES
2,SalesQuota,money,YES
3,Bonus,money,NO
4,CommissionPct,smallmoney,NO
5,SalesYTD,money,NO
6,SalesLastYear,money,NO
7,rowguid,uniqueidentifier,NO
8,ModifiedDate,datetime,NO


Extracting data from database with specific params using `select` method

In [19]:
df = sql.select(table='SalesPerson', columns='Bonus, SalesYTD, SalesLastYear', condition='where Bonus < 4000', schema='Sales')
df.head()

,Bonus,SalesLastYear,SalesYTD
0,0.0000,0.0000,559697.5639
1,2000.0000,1439156.0291,4251368.5497
2,2500.0000,1997186.2037,3189418.3662
3,500.0000,1620276.8966,1453719.4653
4,3550.0000,2073505.9999,2458535.6169


Exporting a specific table from database as csv format using `export_to_file` method

We used as example a specific single table, however, `tables` param supports also a list of tables to be passed. If no tables are passed, it will be setted as Nonetype (default) and all database tables will be exported.

It's also possible to export tables from another database without needing to manually access the database with `set_database` method. 

Example: 

    sql.export_to_file(tables=['table1', 'table2', ... 'tableN'], database='Desired Database')

`database` by default is 'current'

Moreover, the following file formats are also supported: xlsx, json and txt.

If `file_type` param is passed as __json__, you can also pass json orientation through `json_orient` param

In [10]:
sql.export_to_file(tables='Illustration', format_='csv')

Error while executing query, attempting to query table with schema


Checking the data we just exported

In [11]:
Illustration = pd.read_csv('Illustration.csv')
Illustration

,Unnamed: 0,Diagram,IllustrationID,ModifiedDate
0,0,<!-- Generated by Adobe Illustrator CS -> XAML...,3,2014-01-09 13:06:11.780
1,1,<!-- Generated by Adobe Illustrator CS -> XAML...,4,2014-01-09 13:06:11.903
2,2,<!-- Generated by Adobe Illustrator CS -> XAML...,5,2014-01-09 13:06:11.950
3,3,<!-- Generated by Adobe Illustrator CS -> XAML...,6,2014-01-09 13:06:12.043
4,4,<!-- Generated by Adobe Illustrator CS -> XAML...,7,2014-01-09 13:06:12.080


Essentially, `query` method was designed as the most important method. Its the function that actually uses pyodbc connection to interacts with database. It's internally called on pretty much every function demonstrated in this notebook so far.

However, it can be used to perform other more specific taskes which are not contemplated by other functions, here we create a new table.

NOTE: 

Whenever a transaction must be done, use `commit` param as True. If commit is true, this function understands that a transaction must be made, so no data will be returned. If commit is False (default), it understands that some data is expected to be outputed, and return_option ('raw', 'single value' or 'list') param will take action.


In [31]:
create_tb = 'create table tb_test (col_one int not null, col_two int not null, col_three varchar(10) not null)'
sql.query(create_tb, commit=True)

Checks if the table has been properly created

In [32]:
sql.detail_table('tb_test')

,COLUMN_NAME,DATA_TYPE,IS_NULLABLE
0,col_one,int,NO
1,col_two,int,NO
2,col_three,varchar,NO


As expected table is empty, therefore no dataframe can be constructed from it

In [18]:
sql.select('tb_test')

No data returned from query, returning None


Import pandas and crate a new dataframe so that we can insert some values into our created table

In [12]:
import pandas as pd
insert_dataframe = pd.DataFrame({'col_one':[1,2,3,4], 
                                 'col_two':[3,5,6,6], 
                                 'col_three':['a','b','c','d']})
insert_dataframe

,col_one,col_three,col_two
0,1,a,3
1,2,b,5
2,3,c,6
3,4,d,6


Using `insert` method to add our data into __tb_test__. 

NOTE:

For now, this function is very simple and there isn't any procedure to validate if input dataframe columns matches perfectly sql table's columns (regarding column's name, not position). Ergo, make sure that input dataframe has exactly the same columns as database table otherwise it will throw an error.  

In [13]:
sql.insert(insert_dataframe, 'tb_test')

Checking if data was properly inserted 

In [14]:
sql.select('tb_test')

,col_one,col_three,col_two
0,1,a,3
1,2,b,5
2,3,c,6
3,4,d,6
